# Monte Carlo Methods
In this notebook, you will write your own implementations of many Monte Carlo(MC) algorithms.
While we have provided some starter code, you are welcome to erase these hints and write code from scratch

## Part 0: Explore BlackjackEnv
We begin by importing the necessary packages.

In [1]:
import sys
import gym
import numpy as np
from collections import defaultdict

from plot_utils import plot_blackjack_values, plot_policy

Use the code cell below to create an instance of the Blackjack environment.